In [1]:
# 20200612 JLCY

from netCDF4 import Dataset
import glob
import wrf
import xarray as xr

## extract WRF data (from Eagle)

In [ ]:
target_height = [ 40.,  50.,  60.,  80., 100., 120., 140., 160., 180., 200.] # agl

lat_start, lat_end = 410, 560
lon_start, lon_end = 390, 510

wrf_target_dir = '/Users/jlee/eagle/wfip2_wrf_from_eagle/'
output_dir = '/Users/jlee/eagle/wfip2_wrf_py/'

In [2]:
wrf_file_list = glob.glob(wrf_target_dir+'*')
wrf_file_list.sort()

In [3]:
wrf_file_list[0]

'/Users/jlee/eagle/wfip2_wrf_from_eagle/wrfout_d02_2016-09-23_12:00:00'

In [8]:
for file in wrf_file_list: 

    wrf_data = Dataset(file, 'r')

    ua = wrf.getvar(wrf_data, 'ua')
    va = wrf.getvar(wrf_data, 'va')
    ght = wrf.g_geoht.get_height(wrf_data, msl=False)

    ua_subset = ua[:, lat_start:lat_end, lon_start:lon_end]
    va_subset = va[:, lat_start:lat_end, lon_start:lon_end]
    ght_subset = ght[:, lat_start:lat_end, lon_start:lon_end]

    ua_ght = wrf.interplevel(ua_subset, ght_subset, target_height)
    va_ght = wrf.interplevel(va_subset, ght_subset, target_height)

    del ua_ght.attrs['projection']
    del va_ght.attrs['projection']

    ds = xr.Dataset({'u': ua_ght, 'v': va_ght})
    ds.to_netcdf(output_dir+file.split('/')[-1]+'.nc')

## SODAR data

In [13]:
sodar_target_dir = '/Users/jlee/eagle/wfip2_sodar_from_eagle/'

# 10-minute files
sodar_file_list = glob.glob(sodar_target_dir+'*')

sodar_data = Dataset(sodar_file_list[0], 'r')

In [49]:
sodar_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2017-04-13 05:42:20 created by libingest-1.0 using wfip2_sodar-1.0 (build version: v1.1.0)
    dimensions(sizes): time(1), height(10), bounds(2)
    variables(dimensions): float64 time(time), float64 time_bounds(time,bounds), float32 height(height), float32 wind_speed(time,height), float32 wind_direction(time,height), float32 vertical_air_velocity(time,height), float64 latitude(), float64 longitude(), float64 altitude()
    groups: 

In [50]:
sodar_data['latitude'][0]

masked_array(data=45.57451,
             mask=False,
       fill_value=1e+20)

In [51]:
sodar_data['longitude'][0]

masked_array(data=-120.74734,
             mask=False,
       fill_value=1e+20)

In [52]:
sodar_data['time'][0]

masked_array(data=0.,
             mask=False,
       fill_value=1e+20)

In [63]:
sodar_data['wind_speed'][:].data

array([[2.82000e+00, 2.93000e+00, 2.88000e+00, 2.48000e+00, 3.21000e+00,
        2.74000e+00, 3.11000e+00, 3.36000e+00, 2.59000e+00, 9.96921e+36]],
      dtype=float32)

In [64]:
sodar_data['wind_speed']

<class 'netCDF4._netCDF4.Variable'>
float32 wind_speed(time, height)
    long_name: wind speed
    standard_name: wind_speed
    units: m/s
    cell_methods: time: mean
unlimited dimensions: time
current shape = (1, 10)
filling on, default _FillValue of 9.969209968386869e+36 used

In [16]:
sodar_data['height'][:].data

array([ 40.,  50.,  60.,  80., 100., 120., 140., 160., 180., 200.],
      dtype=float32)